# Applied Data Analysis Project
**Team**: ToeStewBrr - Alexander Sternfeld, Marguerite Thery, Antoine Bonnet, Hugo Bordereaux

**Dataset**: CMU Movie Summary Corpus


In [ ]:
import requests
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from nltk.tree import Tree
import xml.etree.ElementTree as ET
import itertools
import matplotlib.dates as mdates

from load_data import *
pd.options.mode.chained_assignment = None


## 1. Loading data

We first extract all files from the [MoviesSummaries dataset](http://www.cs.cmu.edu/~ark/personas/). 

`corenlp_plot_summaries.tar.gz [628 M, separate download]`: The plot summary of each movie, run through the Stanford CoreNLP pipeline (tagging, parsing, NER and coref). Each filename begins with the Wikipedia movie ID (which indexes into movie.metadata.tsv).

We extract all coreNLP files, then uncompress them to the XML format. 

Note: Extraction of CoreNLP files takes 15 minutes, while conversion takes 30 seconds. 

In [ ]:
download_data()

## 2. Pre-processing data

### 2.1. Plot summaries

`plot_summaries.txt [29 M]`: Plot summaries of 42,306 movies extracted from the November 2, 2012 dump of English-language Wikipedia.  Each line contains the Wikipedia movie ID (which indexes into movie.metadata.tsv) followed by the summary.

In [ ]:
plot_df = load_plot_df()
plot_df

In [ ]:
# For Hugo: this method stems the words to their lexical root. 
# Implement Stemming using out of the box Porter algorithm
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
plot_stemmed = [[stemmer.stem(word) for word in sentence.split(" ")] for sentence in plot_df.iloc[:5].Summary]
plot_stemmed = [" ".join(sentence) for sentence in plot_stemmed]


### 2.2. Movie metadata

`movie.metadata.tsv.gz [3.4 M]`: Metadata for 81,741 movies. 

In [ ]:
movie_df = load_movie_df()
movie_df

### 2.3. Character metadata

`character.metadata.tsv.gz [14 M]`: Metadata for 450,669 characters aligned to the movies above. 


In [ ]:
char_df = load_char_df()
char_df

### 2.4. Name clusters

`name.clusters.txt`: 970 unique character names used in at least two different movies, along with 2,666 instances of those types.  The ID field indexes into the Freebase character/actor map ID in character.metadata.tsv.

In [ ]:
names_df = load_names_df()
names_df

### 2.5. TV Tropes Clusters

`tvtropes.clusters.txt`: 72 character types drawn from tvtropes.com, along with 501 instances of those types.  The ID field indexes into the Freebase character/actor map ID in character.metadata.tsv.

In [ ]:
cluster_df = load_cluster_df()
cluster_df


We now join the TV tropes clusters with movie.metadata so we are able to access movie genre and filter on romance. 

In [ ]:
cluster_char = cluster_df.merge(char_df, on='Freebase character/map ID')
cluster_char_movie = cluster_char.merge(movie_df, on='Freebase ID')
romance_cluster = cluster_char_movie[cluster_char_movie['Genres'].apply(lambda x: any('Roman' in genre for genre in x))]
romance_cluster.groupby(romance_cluster['Cluster']).size().sort_values(ascending=False)
romance_cluster

### 2.6. Missing values and outliers

#### 2.6.1. Identifying missing values

In [ ]:
# Missing values
movie_cols = ['Wikipedia ID', 'Freebase ID', 'Name', 'Release date', 
'Box office revenue', 'Runtime', 'Languages', 'Countries', 'Genres']
for column in movie_cols[3:]:
    number_missing = movie_df[column].isna().sum()
    print(f'{column}: {number_missing}/{len(movie_df)} ({round(100*number_missing/movie_df.shape[0],2)}%) missing values')


#### 2.6.2. Outliers
- We note that there is one movie with a release date in the year 1010, we manually correct this to the date 2010
- There are many movies with very short runtimes (under 5 minutes). We manually verified a sample of these movies, and have found that the runtimes are correct.
- There are over a hundred movies with runtimes over 5 hours. We have found that most of these data points correspond to series, where the runtime is the sum of the episode lengths.
- One movie has a runtime of over a million minutes ('Zero tolerance'). This lengths is manually corrected to 88 minutes.

In [ ]:
# Isolate the year from the release date
getYear = lambda x: x[slice(0,4)] if type(x) == str else x
movie_df['Release date'] = movie_df['Release date'].apply(getYear)

# Identify suspicious movie with release date 1010, manually correct
outlier_release_date = movie_df[movie_df['Release date'].apply(lambda x: int(x.split('-')[0]) < 1850 if type(x)==str else False)]
movie_df['Release date'][movie_df['Wikipedia ID']==29666067] = '2010'

# Identify suspicious movies with long runtimes
movie_df[movie_df['Runtime'] > 300]
movie_df['Runtime'][movie_df['Wikipedia ID']==10815585] = '88'


## 3. Exploratory Data Analysis


### 3.1. Analysing romantic genres

One notices that there are several types of romantic movies: romantic comedy, romance film, romantic drama. 

In [ ]:
romance_genres = ['Romantic comedy', 'Romance Film', 'Romantic drama', 'Romantic fantasy', 'Romantic thriller']
is_romantic = lambda i: lambda x: any(y in romance_genres[i] for y in x) if type(x) == list else False
is_not_romantic = lambda i: lambda x: not any(y in romance_genres[i] for y in x) if type(x) == list else False
romance_movies = movie_df[movie_df['Genres'].apply(is_romantic(slice(0, 5)))]
non_romance_movies = movie_df[movie_df['Genres'].apply(is_not_romantic(slice(0, 5)))]

In [ ]:
#Organize by category
romantic_comedy = romance_movies.loc[movie_df['Genres'].apply(is_romantic(0))]
romantic_film = romance_movies.loc[movie_df['Genres'].apply(is_romantic(1))]
romantic_drama = romance_movies.loc[movie_df['Genres'].apply(is_romantic(2))]
romantic_fantasy = romance_movies.loc[movie_df['Genres'].apply(is_romantic(3))]
romantic_thriller = romance_movies.loc[movie_df['Genres'].apply(is_romantic(4))]
print('Romance:' , romance_movies.shape[0])
print('Romantic comedies: ', romantic_comedy.shape[0], '\nRomantic films: ', romantic_film.shape[0], '\nRomantic drama: ', romantic_drama.shape[0], '\nRomantic fantasy: ', romantic_fantasy.shape[0], '\nRomantic thriller: ', romantic_thriller.shape[0])
print('Total number of films: ', movie_df.shape[0])

### 3.2. Romantic movies runtime

In [ ]:
ax = sns.kdeplot(romantic_comedy['Runtime'], color='blue')
ax = sns.kdeplot(romantic_drama['Runtime'], color='green')
ax = sns.kdeplot(romantic_film['Runtime'], color='red')
ax = sns.kdeplot(romantic_fantasy['Runtime'], color='orange')
ax = sns.kdeplot(non_romance_movies[non_romance_movies['Runtime'] < 200]['Runtime'], color = 'purple')
ax.set_xlim(0, 200)
ax.legend(['Romantic comedy', 'Romantic drama', 'Romance Film', 'Romantic fantasy', 'Non romantic movies'])
plt.title('Movie runtime distribution by genre')
plt.xlabel('Runtime (minutes)')
plt.ylabel('Density')
plt.show()

### 3.3. Romantic movies box office revenue

In [ ]:
ax = sns.kdeplot(romantic_comedy['Box office revenue'], log_scale=True, color='blue')
ax = sns.kdeplot(romantic_drama['Box office revenue'], log_scale=True, color='green')
ax = sns.kdeplot(romantic_film['Box office revenue'], log_scale=True, color='red')
ax = sns.kdeplot(romantic_fantasy['Box office revenue'], log_scale=True, color='orange')
ax = sns.kdeplot(non_romance_movies['Box office revenue'], log_scale=True, color='purple')
ax.legend(['Romantic comedy', 'Romantic drama', 'Romance Film', 'Romantic fantasy', 'Non romantic movies'])
plt.title('Box office revenue distribution by genre')
plt.xlabel('Revenue (USD)')
plt.ylabel('Density')
plt.show()

### 3.4. Romantic movies countries

In [ ]:
get_countries = lambda x: len(x) if type(x) == list else np.nan
romantic_comedy['number_countries'] = romantic_comedy['Countries'].apply(get_countries)
romantic_fantasy['number_countries'] = romantic_fantasy['Countries'].apply(get_countries)
romantic_film['number_countries'] = romantic_film['Countries'].apply(get_countries)
romantic_drama['number_countries'] = romantic_drama['Countries'].apply(get_countries)

combined_numb_countries = pd.DataFrame({
    'Romantic comedy': romantic_comedy['number_countries'], 
    'Romance Film': romantic_film['number_countries'], 
    'Romantic drama': romantic_drama['number_countries'], 
    'Romantic fantasy': romantic_fantasy['number_countries']})

print('Percentage romantic comedy movie countries > 1: ', round(romantic_comedy[romantic_comedy['number_countries']> 1].shape[0]/romantic_comedy.shape[0], 2), '%')
print('Other countries can be added in code...')

### 3.5. Movie languages

In [ ]:
#Get languages whole movie set
movies_language = movie_df[movie_df['Languages'].notnull()]
languages=movies_language['Languages'].sum()
values, counts = np.unique(languages, return_counts=True)
print('5 most common languages in movies are: ')
print(values[counts.argsort()[-5:][::-1]])

#Get languages romantic movies overall
romance_movies_lang = romance_movies[romance_movies['Languages'].notnull()]
languages_romance = romance_movies_lang.Languages.sum()
values, counts = np.unique(languages_romance, return_counts=True)
print('\n5 most common languages in romantic movies: ')
print(values[counts.argsort()[-5:][::-1]])


rom_com_known = romantic_comedy[romantic_comedy['Languages'].notnull()]
languages_romcom = rom_com_known.Languages.sum()
values, counts = np.unique(languages_romcom, return_counts=True)
print('\n5 most common languages in romantic comedies: ')
print(values[counts.argsort()[-5:][::-1]])

### 3.6. Evolution over time

#### 3.6.1 Box office revenue over time

In [ ]:
year_box_office = non_romance_movies[['Release date', 'Box office revenue']].dropna()
year_box_office_romance = romance_movies[['Release date', 'Box office revenue']].dropna()
year_box_office['Release date'] = pd.to_datetime(year_box_office['Release date'])
year_box_office_romance['Release date'] = pd.to_datetime(year_box_office_romance['Release date'])

In [ ]:
import matplotlib
# Get yearly box office revenue for romantic and non-romantic movies
year_box_office = non_romance_movies[['Release date', 'Box office revenue']].dropna()
year_box_office_romance = romance_movies[['Release date', 'Box office revenue']].dropna()
year_box_office['Release date'] = pd.to_datetime(year_box_office['Release date'])
year_box_office_romance['Release date'] = pd.to_datetime(year_box_office_romance['Release date'])

# TODO: Fix the standard deviations for the box office revenue
total_mean = year_box_office.groupby('Release date').mean().iloc[1:]
romance_mean = year_box_office_romance.groupby('Release date').mean().iloc[1:]
total_std = year_box_office.groupby('Release date').std().replace(np.nan, 0).iloc[1:]/10
romance_std = year_box_office_romance.groupby('Release date').std().replace(np.nan, 0).iloc[1:]/10


# Plot average monthly revenue for romance movies and non romance movies
fig, ax = plt.subplots(figsize=(15, 7))
ax.plot(total_mean.index, total_mean['Box office revenue'], color='blue', label='Non romance movies')
ax.plot(romance_mean.index, romance_mean['Box office revenue'], color='red', label='Romance movies')
ax.fill_between(total_mean.index, total_mean['Box office revenue'] - total_std['Box office revenue'], total_mean['Box office revenue'] + total_std['Box office revenue'], color='blue', alpha=0.2)
ax.fill_between(romance_mean.index, romance_mean['Box office revenue'] - romance_std['Box office revenue'], romance_mean['Box office revenue'] + romance_std['Box office revenue'], color='red', alpha=0.2)
years = mdates.YearLocator(5)
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('${x:,.0f}'))
ax.set_xlabel('Release date')
ax.set_ylabel('Revenue (USD)')
ax.set_title('Average monthly revenue for romance movies and non romance movies')
ax.legend()
plt.show()


### 3.6.2. Evolution of runtime

In [ ]:
import matplotlib

# Get yearly runtime for romantic and non-romantic movies
year_runtime = non_romance_movies[['Release date', 'Runtime']].dropna()
year_runtime_romance = romance_movies[['Release date', 'Runtime']].dropna()
year_runtime['Runtime'] = year_runtime['Runtime'].astype(float)
year_runtime_romance['Runtime'] = year_runtime_romance['Runtime'].astype(float)

# TODO: Fix the standard deviations for the runtime
total_runtime_mean = year_runtime.groupby('Release date').mean()
romance_runtime_mean = year_runtime_romance.groupby('Release date').mean()
total_runtime_std = year_runtime.groupby('Release date').std().replace(np.nan, 0)/10
romance_runtime_std = year_runtime_romance.groupby('Release date').std().replace(np.nan, 0)/10

# Convert release date of mean and std to datetime
total_runtime_mean.index = pd.to_datetime(total_runtime_mean.index)
romance_runtime_mean.index = pd.to_datetime(romance_runtime_mean.index)
total_runtime_std.index = pd.to_datetime(total_runtime_std.index)
romance_runtime_std.index = pd.to_datetime(romance_runtime_std.index)


# Plot average monthly runtime for romance movies and non romance movies
fig, ax = plt.subplots(figsize=(15, 7))
ax.plot(total_runtime_mean.index, total_runtime_mean['Runtime'], color='blue', label='Non romance movies')
ax.plot(romance_runtime_mean.index, romance_runtime_mean['Runtime'], color='red', label='Romance movies')
ax.fill_between(total_runtime_mean.index, total_runtime_mean['Runtime'] - total_runtime_std['Runtime'], total_runtime_mean['Runtime'] + total_runtime_std['Runtime'], color='blue', alpha=0.2)
ax.fill_between(romance_runtime_mean.index, romance_runtime_mean['Runtime'] - romance_runtime_std['Runtime'], romance_runtime_mean['Runtime'] + romance_runtime_std['Runtime'], color='red', alpha=0.2)
years = mdates.YearLocator(5)
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax.set_xlabel('Release date')
ax.set_ylabel('Runtime (minutes)')
ax.set_title('Average monthly runtime for romance movies and non romance movies')
ax.legend()
plt.show()